In [1]:
import pandas as pd
df = pd.read_csv('binned_df.csv')
print(df.head())

   Unnamed: 0  MAP  diastolic_bp  systolic_bp  urine  ALT  AST  PO2  \
0           0    0             1            2      2    0    0    1   
1           1    0             1            2      2    0    0    2   
2           2    0             1            2      2    0    0    1   
3           3    0             1            1      2    0    0    2   
4           4    0             2            1      2    1    0    0   

   lactic_acid  serum_creatinine  FiO2  GCS_total  fluid_boluses  \
0            0                 0     2          2              0   
1            0                 0     2          2              0   
2            0                 0     2          2              0   
3            0                 0     2          2              0   
4            0                 0     2          2              0   

   vasopressors  SOFA_score  PatientID  Timepoints  
0             0           4          0           0  
1             0           3          0           1  
2    

In [5]:
state_columns = ['MAP', 'diastolic_bp', 'systolic_bp', 'urine', 'ALT', 'AST', 'PO2', 'lactic_acid', 'serum_creatinine', 'FiO2', 'GCS_total']
action_columns = ['fluid_boluses', 'vasopressors']

In [12]:
#reward function: SOFA score measures patient health (higher score/increase in score should be penalised)
#referenced from: Deep Reinforcement Learning for Sepsis Treatment
def calculate_reward(cur_score, next_score):
    reward = 0
    c0 = -0.025
    c1 = -0.125
    if cur_score == next_score and next_score > 0:
        reward += c0
    return reward + c1 * (next_score - cur_score)

#calculate SOFA score for each state
def calculate_sofa_score(MAP, urine, ALT, AST, PO2, lactic_acid, serum_creatinine, FiO2, GCS_total):
    cardiovascular_score = 0
    if MAP < 70:
        cardiovascular_score = 1
    
    respitory_score = 0
    if FiO2 > 0:
        p_f_ratio = PO2 / FiO2
        
        if p_f_ratio < 100:
            respitory_score = 4
        elif p_f_ratio < 200:
            respitory_score = 3
        elif p_f_ratio < 300:
            respitory_score = 2
        else:
            respitory_score = 1
    
    renal_score = 0
    if serum_creatinine >= 1.2 and serum_creatinine < 2:
        renal_score = 1
    elif serum_creatinine >= 2 and serum_creatinine < 3.5:
        renal_score = 2
    elif serum_creatinine >= 3.5 and serum_creatinine < 5:
        renal_score = 3
    elif serum_creatinine >= 5.0:
        renal_score = 4
    #divide daily urine output standards by 6 as the data is 4 hour interval
    if urine < 500.0 / 6:
        renal_score = max(renal_score, 3)
    if urine < 200.0 / 6:
        renal_score = max(renal_score, 4)

    #since bilirubin is not available, use ALT and AST to calculate liver score
    liver_score = 0
    if (ALT > 40 and AST > 40):
        liver_score = 1
    if (ALT > 120 or AST > 120):
        liver_score = 2
    if (ALT > 300 or AST > 300):
        liver_score = 3
    if (ALT > 1000 or AST > 1000):
        liver_score = 4

    neuro_score = 0
    if GCS_total < 6:
        neuro_score = 4
    elif GCS_total < 10:
        neuro_score = 3
    elif GCS_total < 13:
        neuro_score = 2
    elif GCS_total < 15:
        neuro_score = 1
    
    lactic_acid_score = 0
    if lactic_acid > 2:
        lactic_acid_score = 2
    
    return cardiovascular_score + respitory_score + renal_score + liver_score + neuro_score + lactic_acid_score

In [ ]:
#for mdptoolbox Value Iteration, the reward function is (S, A), it doesn't use next state
#this method only works with R(S, A) reward function
import mdptoolbox
import numpy as np

states = df[['PatientID', 'Timepoints']].values
num_states = len(states)
num_actions = 25
P = np.zeros((num_actions, num_states, num_states))
R = np.zeros((num_states, num_actions))

for i in range(len(df) - 1):
    if df.iloc[i]['PatientID'] == df.iloc[i + 1]['PatientID'] and df.iloc[i]['Timepoints'] + 1 == df.iloc[i + 1]['Timepoints']:
        current_state = i
        next_state = i + 1
        action = df.iloc[i]['ActionID']

        reward = calculate_reward(df.iloc[i]['SOFA_score'], df.iloc[i + 1]['SOFA_score'])

        P[action, current_state, next_state] += 1

        R[current_state, action] = reward

for action in range(num_actions):
    for state in range(num_states):
        if P[action, state].sum() > 0:
            P[action, state] /= P[action, state].sum()

vi = mdptoolbox.mdp.ValueIteration(P, R, 0.9)
vi.run()

print("Optimal Policy:", vi.policy)
print("Optimal Value Function:", vi.V)

In [10]:
from collections import defaultdict
import numpy as np

num_states = len(df[state_columns].value_counts())
num_actions = len(df[action_columns].value_counts())
Q = defaultdict(lambda: np.zeros((num_states, num_actions)))
print(Q[0][0])

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [2]:
from sklearn.model_selection import train_test_split

# Split the data
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [17]:
from collections import defaultdict
import random

class DynaQAgent:
    def __init__(self, state_size, action_size, gamma=0.9, alpha=0.1, epsilon=0.1, planning_steps=10):
        self.state_size = state_size
        self.action_size = action_size
        self.gamma = gamma
        self.alpha = alpha
        self.epsilon = epsilon
        self.planning_steps = planning_steps
        self.q_table = np.zeros((state_size, action_size))
        self.model = defaultdict(lambda: (None, 0))  #(next_state, reward)

    def choose_action(self, state):
        if np.random.rand() < self.epsilon:
            return np.random.randint(self.action_size)
        else:
            return np.argmax(self.q_table[state])

    def update(self, state, action, reward, next_state):
        best_next_action = np.argmax(self.q_table[next_state])
        td_target = reward + self.gamma * self.q_table[next_state, best_next_action]
        self.q_table[state, action] += self.alpha * (td_target - self.q_table[state, action])

        self.model[(state, action)] = (next_state, reward)

        for _ in range(self.planning_steps):
            sim_state, sim_action = random.choice(list(self.model.keys()))
            sim_next_state, sim_reward = self.model[(sim_state, sim_action)]

            sim_best_next_action = np.argmax(self.q_table[sim_next_state])
            sim_td_target = sim_reward + self.gamma * self.q_table[sim_next_state, sim_best_next_action]
            self.q_table[sim_state, sim_action] += self.alpha * (sim_td_target - self.q_table[sim_state, sim_action])


In [6]:
state_dict = {tuple(row): idx for idx, row in enumerate(df[state_columns].drop_duplicates().values)}
action_dict = {tuple(row): idx for idx, row in enumerate(df[action_columns].drop_duplicates().values)}

def get_state_index(row):
    return state_dict[tuple(row[state_columns])]

def get_action_index(row):
    return action_dict[tuple(row[action_columns])]


In [29]:
print(action_dict)

{(0, 0): 0, (2, 0): 1, (1, 0): 2, (0, 1): 3, (0, 4): 4, (4, 0): 5, (2, 1): 6, (1, 1): 7, (4, 4): 8, (2, 4): 9, (4, 1): 10, (1, 4): 11}


In [21]:
state_size = len(state_dict)
action_size = len(action_dict)
print(state_size)
print(action_size)

agent = DynaQAgent(state_size, action_size)

i = 0
for idx, row in train_df.iterrows():
    state = get_state_index(row)
    action = get_action_index(row)
    if i % 1000 == 0:
        print(i)
    i += 1
    next_state_row = train_df[(train_df['PatientID'] == row['PatientID']) & (train_df['Timepoints'] == row['Timepoints'] + 1)]
    
    if not next_state_row.empty:
        next_state = get_state_index(next_state_row.iloc[0])
        reward = calculate_reward(row['SOFA_score'], next_state_row.iloc[0]['SOFA_score'])
        
        agent.update(state, action, reward, next_state)


9109
12
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000


In [27]:
cumulative_reward = 0

for idx, row in test_df.iterrows():
    state = get_state_index(row)
    action = agent.choose_action(state)

    next_state_row = test_df[(test_df['PatientID'] == row['PatientID']) & (test_df['Timepoints'] == row['Timepoints'] + 1)]
    
    if not next_state_row.empty:
        next_state = get_state_index(next_state_row.iloc[0])
        reward = calculate_reward(row['SOFA_score'], next_state_row.iloc[0]['SOFA_score'])
        
        cumulative_reward += reward

print("total cumulative reward (Dyna-Q) with SOFA score reward:", cumulative_reward)

total cumulative reward (Dyna-Q) with SOFA score reward: -48.29999999999833


In [32]:
def calculate_cumulative_reward(df):
    cumulative_rewards = 0

    for patient_id, patient_data in df.groupby('PatientID'):
        patient_data = patient_data.sort_values('Timepoints')
        total_reward = 0

        for i in range(len(patient_data) - 1):
            current_state = patient_data.iloc[i]
            next_state = patient_data.iloc[i + 1]
            reward = calculate_reward(current_state['SOFA_score'], next_state['SOFA_score'])
            total_reward += reward

        cumulative_rewards += total_reward

    return cumulative_rewards

cumulative_rewards = calculate_cumulative_reward(test_df)
print("total cumulative reward (clinician) with SOFA score reward:", cumulative_rewards)

total cumulative reward (clinician) with SOFA score reward: -70.92499999999883


In [33]:
def no_action_cumulative_reward(states_df, transition_probs, initial_state, num_steps=100):
    current_state = initial_state
    cumulative_reward = 0

    for step in range(num_steps):
        current_index = states_df.index[states_df == current_state].tolist()[0]
        next_state_index = np.random.choice(len(states_df), p=transition_probs[current_index, :, 0])
        next_state = states_df.iloc[next_state_index]

        reward = calculate_reward(current_state, next_state)
        cumulative_reward += reward

        current_state = next_state

    return cumulative_reward
